In [1]:
#!pip install text2emotion # Install of text2emotion, unneccessary for webapp hosteed

In [2]:
from tensorflow.keras import models # Import models for laoding and prediction
from tensorflow.keras.preprocessing.sequence import pad_sequences # Import to pad sequences to reshape data
import numpy as np # Import numpy for list management
import text2emotion as te # Import text2emotion for emotion classificaiton of inputs
import re # Import regular expressions for cleaning text
import pickle # Import pickle for file opening

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
def ChatbotInputHandling(userInput):  # Funciton to take user input and process for response   
    if userInput.startswith('my name is'): # Rules based implementation to handle name 
        name = userInput[11:] # Takes user name from their input
        chatbotOutput = (f"Hi {name}, I am a chatbot designed to offer emotional support and informational support about Edge Hill University. Try asking me \"where is the catalyst \".") # Responds with introduction
    elif userInput.startswith('hi my name is'): # Rules based implementation to handle name 
        name = userInput[14:] # Takes user name from their input
        chatbotOutput = (f"Hello {name}, I am a chatbot designed to offer emotional support and informational support about Edge Hill University. Try asking me \"where is the catalyst \".") # Responds with introduction
    else: # If not any rule matched
        processedInput = TextToTokens(userInput) # Processes user input
        if (processedInput[0][0] == 0): # If the chatbot can't recognise what was said
            chatbotOutput = (f"Sorry I didn't catch what you said") # Prints default message
        else:
            statesValues = encoderModel.predict(processedInput) # Takes input and uses encoder model to find the states for the input
            targetSequence = np.zeros((1, 1)) # Defines an empty list for holding the previous state
            targetSequence[0, 0] = wordIndex['bos'] # Initilisaes with the start marker
            stopCondition = False # Sets boolean to handle stopping the iteration
            chatbotOutput = '' # Empty string for holding the predicted text
            while not stopCondition: # While not stopping
                decoderOutputs, h, c = decoderModel.predict([targetSequence] + statesValues) # Use states and previous state to predict word weightings
                sampledWordIndex = np.argmax(decoderOutputs[0, -1, :]) # Finds the locaiton of the most predicted word
                sampledWord = None # Originally defines a word sample as none
                if sampledWordIndex in indexWord: # If the location exsits in the reversed index
                    word = indexWord[sampledWordIndex] # Word is made the value for said location
                    if word != 'eos': # If that word is not the end marker
                        chatbotOutput += f' {word}' # Add the word to the translation
                    sampledWord = word # sets word to the sample word for future use
            
                    if sampledWord == 'eos' or len(chatbotOutput.split()) > decoderMaxLength: # If the end marker is met or the max length is met
                        stopCondition = True # Stop predicting
            
                    targetSequence = np.zeros((1, 1)) # Emptys target liust
                    targetSequence[0, 0] = sampledWordIndex # Makes the list the last sampled word's index
                    statesValues = [h, c] # Sets the states values to those found in the decoder iteration
    
    return chatbotOutput # returns the output to the app the output

In [4]:
def TextToTokens(text: str): # Function for cleaning inputs
    currentEmotionList = list() # Creates empty list for current emotions
    emotionDictionary = te.get_emotion(text) # Establishes an emotion dictionary of input
    for emotion in emotionDictionary: # For each emotion in said dictiionary
        currentEmotionList.append(emotionDictionary[emotion]) # Add to the current input list

    text = CleanText(text) # Calls existing cleaning method from base
    words = text.split() # Splits text into words
    tokensList = list() # Makes list for each token
    for word in words: # For every word
        result = wordIndex.get(word,"") # Gets the index, nothing if not existing
        if result != "": # If result isnt nothing
             tokensList.append(result) # Adds to the token list
    tokensList += currentEmotionList # Adds the lists together
    return pad_sequences([tokensList],maxlen=encoderMaxLength,padding='post') # Returns the token list as padded to match encoder length for use in encoder

In [5]:
def CleanText(text): # Function for cleaning text, removing unneccesary characters and making lower case
        
    if type(text) == dict: # Makes sure text is not a dictionary (Common error due to : use in text)
        keys = list(text) # Gets the key (note there should only be one )
        totalText = "" # Empty to hold the entire text
        for key in keys: # For each key
            value = text[key] # Finds the value associaaed 
            text = f"{key} + {value}" # Appends data back together without colon
            totalText = totalText + text # adds to total string
        text = totalText # makes text the total string
    text = text.lower() # Converts text input to lowercase
    text = re.sub(r"i'm", "i am", text) # Removes potentially conflicting punctuation, swapping with full text
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text) # Removes conflicting characters completely
        
    return text # Return cleaned text

In [12]:
def PickleOpen(fileName): # Function to open items with picckle
    with open(f'{fileName}.pickle', 'rb') as f: # Opens directory
        item = pickle.load(f) # loads item
        return item # Returns item

In [13]:
encoderModel = models.load_model("encoderModel.h5") # Load chatbot data
decoderModel = models.load_model("decoderModel.h5")
decoderMaxLength = PickleOpen("decoderMaxLength")
encoderMaxLength = PickleOpen("encoderMaxLength")
wordIndex = PickleOpen("wordIndex")
indexWord = PickleOpen("indexWord")